In [27]:
import json
import hashlib
from algosdk.v2client import algod
from algosdk import account, mnemonic
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn, wait_for_confirmation
from algosdk import account
import algosdk

Create Asset

In [28]:
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
algod_address = "http://192.168.137.236:4001"
algod_client = algod.AlgodClient(algod_token, algod_address)


In [29]:
def print_created_asset(algodclient, account, assetid):
  # note: if you have an indexer instance available it is easier to just use this
  # response = myindexer.accounts(asset_id = assetid)
  # then use 'account_info['created-assets'][0] to get info on the created asset
  account_info = algodclient.account_info(account)
  idx = 0
  for my_account_info in account_info['created-assets']:
    scrutinized_asset = account_info['created-assets'][idx]
    idx = idx + 1       
    if (scrutinized_asset['index'] == assetid):
      print("Asset ID: {}".format(scrutinized_asset['index']))
      print(json.dumps(my_account_info['params'], indent=4))
      break

#   Utility function used to print asset holding for account and assetid
def print_asset_holding(algodclient, account, assetid):
    # note: if you have an indexer instance available it is easier to just use this
    # response = myindexer.accounts(asset_id = assetid)
    # then loop thru the accounts returned and match the account you are looking for
    account_info = algodclient.account_info(account)
    idx = 0
    for my_account_info in account_info['assets']:
        scrutinized_asset = account_info['assets'][idx]
        idx = idx + 1        
        if (scrutinized_asset['asset-id'] == assetid):
            print("Asset ID: {}".format(scrutinized_asset['asset-id']))
            print(json.dumps(scrutinized_asset, indent=4))
            break

In [30]:
# accepts the mnemonic
def create_asset(m,receiver_pk):
    # Convert mnemonic to private key
  private_key = algosdk.mnemonic.to_private_key(m)

  # Get the address from the private key
  public_key = algosdk.account.address_from_private_key(private_key)
  # Store the keys in the accounts dictionary
  accounts = {}
  accounts[1] = {}
  accounts[1]['pk'] = public_key
  accounts[1]['sk'] = private_key

  print("--------------------------------------------")
  print("Creating Asset...")
  # CREATE ASSET
  # Get network params for transactions before every transaction.
  params = algod_client.suggested_params()
  # comment these two lines if you want to use suggested params
  # params.fee = 1000
  # params.flat_fee = True
    
  # JSON file
#   dir_path = os.path.dirname(os.path.realpath(__file__))
#   f = open (dir_path + '/NFT/metadata.json', "r")
  f = open ('../NFT_py/NFT/metadata.json', "r")


  
  # Reading from file
  metadataJSON = json.loads(f.read())
  metadataStr = json.dumps(metadataJSON)

  hash = hashlib.new("sha512_256")
  hash.update(b"arc0003/amj")
  hash.update(metadataStr.encode("utf-8"))
  json_metadata_hash = hash.digest()


  # Account 1 creates an asset called latinum and
  # sets Account 1 as the manager, reserve, freeze, and clawback address.
  # Asset Creation transaction
  txn = AssetConfigTxn(
      sender=accounts[1]['pk'],
      sp=params,
      total=1,
      default_frozen=False,
      unit_name="ALICE001",
      asset_name="Alice's Artwork 001",
      manager=accounts[1]['pk'],
      reserve=receiver_pk,
      freeze=None,
      clawback=None,
      strict_empty_address_check=False,
      url="https://path/to/my/asset/details", 
      metadata_hash=json_metadata_hash,
      decimals=0)

  # Sign with secret key of creator
  stxn = txn.sign(accounts[1]['sk'])

  # Send the transaction to the network and retrieve the txid.
  txid = algod_client.send_transaction(stxn)
  print("Asset Creation Transaction ID: {}".format(txid))


  # Wait for the transaction to be confirmed
  confirmed_txn = wait_for_confirmation(algod_client, txid, 4)  
  print("TXID: ", txid)
  print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
  try:
      # Pull account info for the creator
      # account_info = algod_client.account_info(accounts[1]['pk'])
      # get asset_id from tx
      # Get the new asset's information from the creator account
      ptx = algod_client.pending_transaction_info(txid)
      asset_id = ptx["asset-index"]
      print_created_asset(algod_client, accounts[1]['pk'], asset_id)
      print_asset_holding(algod_client, accounts[1]['pk'], asset_id)
  except Exception as e:
      print(e)


In [31]:
# OPT-IN
def opt_in(trainee_m, asset_id):
    # Convert mnemonic to private key
    trainee_sk = algosdk.mnemonic.to_private_key(trainee_m)

    # Get the address from the private key
    trainee_pk = algosdk.account.address_from_private_key(trainee_sk)

    params = algod_client.suggested_params()
    # comment these two lines if you want to use suggested params
    params.fee = 1000
    params.flat_fee = True


    # Use the AssetTransferTxn class to transfer assets and opt-in
    txn = AssetTransferTxn(
        sender=trainee_pk,
        sp=params,
        receiver=trainee_pk,
        amt=0,
        index=asset_id)
    stxn = txn.sign(trainee_sk)
    txid = algod_client.send_transaction(stxn)
    print(txid)
    # Wait for the transaction to be confirmed
    wait_for_confirmation(algod_client, txid)
    # Now check the asset holding for that account.
    # This should now show a holding with a balance of 0.
    print_asset_holding(algod_client, trainee_pk, asset_id)

In [32]:
def transfer_asset(sender_m, receiver_pk, asset_id):
    # Convert mnemonic to private key
    sender_sk = algosdk.mnemonic.to_private_key(sender_m)

    # Get the address from the private key
    sender_pk = algosdk.account.address_from_private_key(sender_sk)

    # transfer asset of 10 from account 1 to account 3
    params = algod_client.suggested_params()
    # comment these two lines if you want to use suggested params
    params.fee = 1000
    params.flat_fee = True
    txn = AssetTransferTxn(
        sender=sender_pk,
        sp=params,
        receiver=receiver_pk,
        amt=1,
        index=asset_id)
    stxn = txn.sign(sender_sk)
    txid = algod_client.send_transaction(stxn)
    print(txid)
    # Wait for the transaction to be confirmed
    wait_for_confirmation(algod_client, txid)
    # The balance should now be 10.
    print_asset_holding(algod_client, receiver_pk, asset_id)

## sender

In [33]:
import os,sys
cwd = os.getcwd()
sys.path.append(f'../NFT_py/')
from create_account import create_account
m=create_account()

My address: ORGL2CWEPC2RRQHGT4YTBTA57ZS3JU4YIGTLNYU7JQKUUZGPUQTM7I2JD4
Account balance: 0 microAlgos

Transaction ID: YCGFNOV4Z2QO5276KASGYQTARRABRQ3JNWA7JFRVUW52JSKJQJSQ
Fund transfer in process...
Fund transferred!
Account balance: 1000000 microAlgos



## receiver

In [34]:
receiver=create_account()

My address: ZYPLVWUNKL4MNZPJNSYFTZYUDJMMDJHGLSGZDUC6NPTBRKMZWUBAYCWXBI
Account balance: 0 microAlgos

Transaction ID: M6AE3THK3F2S4GGKLNZAYOBAOE6UYQMLS6DRJ3JAR72CDRSWQDHA
Fund transfer in process...
Fund transferred!
Account balance: 1000000 microAlgos



In [35]:
print(receiver)

degree oval west bleak arrest hire young noise flash goose kiss inflict spider accident lift laugh midnight behave chaos notable crush help wage absent logic


In [36]:
receiver_pk="ZYPLVWUNKL4MNZPJNSYFTZYUDJMMDJHGLSGZDUC6NPTBRKMZWUBAYCWXBI"

In [37]:
create_asset(m,receiver_pk)

--------------------------------------------
Creating Asset...
Asset Creation Transaction ID: V5H4JU4MBK5BOSITGMSOW65DZQRZ7CYXXMPRUEO4DXMARCQDXIUA
TXID:  V5H4JU4MBK5BOSITGMSOW65DZQRZ7CYXXMPRUEO4DXMARCQDXIUA
Result confirmed in round: 84
Asset ID: 1084
{
    "creator": "ORGL2CWEPC2RRQHGT4YTBTA57ZS3JU4YIGTLNYU7JQKUUZGPUQTM7I2JD4",
    "decimals": 0,
    "default-frozen": false,
    "manager": "ORGL2CWEPC2RRQHGT4YTBTA57ZS3JU4YIGTLNYU7JQKUUZGPUQTM7I2JD4",
    "metadata-hash": "5zAUYEQ+kEKv8VCUNu17DMVNzpzRrF7X9JUr+OEfgp4=",
    "name": "Alice's Artwork 001",
    "name-b64": "QWxpY2UncyBBcnR3b3JrIDAwMQ==",
    "reserve": "ZYPLVWUNKL4MNZPJNSYFTZYUDJMMDJHGLSGZDUC6NPTBRKMZWUBAYCWXBI",
    "total": 1,
    "unit-name": "ALICE001",
    "unit-name-b64": "QUxJQ0UwMDE=",
    "url": "https://path/to/my/asset/details",
    "url-b64": "aHR0cHM6Ly9wYXRoL3RvL215L2Fzc2V0L2RldGFpbHM="
}
Asset ID: 1084
{
    "amount": 1,
    "asset-id": 1084,
    "is-frozen": false
}


In [38]:
asset_id = "1084"
opt_in(receiver,asset_id)

X3Y6F5POAZOUEOKGQ5KVEEIAJF7F2O54HBGUNTBLUZPADTOZUFWQ


In [39]:
transfer_asset(m,receiver_pk,asset_id)

2JTZJS47VKCLFH5CMV5UUGP55MAGQEBIU3KAUAASQGLQKKFBOLEA


In [40]:
create_asset(m,None)

--------------------------------------------
Creating Asset...
Asset Creation Transaction ID: NZCQQLI4UVGP3KG2DWUCC2EMGP22F2NZZLP7VLYNHJSI2MCWTWKQ
TXID:  NZCQQLI4UVGP3KG2DWUCC2EMGP22F2NZZLP7VLYNHJSI2MCWTWKQ
Result confirmed in round: 87
Asset ID: 1087
{
    "creator": "ORGL2CWEPC2RRQHGT4YTBTA57ZS3JU4YIGTLNYU7JQKUUZGPUQTM7I2JD4",
    "decimals": 0,
    "default-frozen": false,
    "manager": "ORGL2CWEPC2RRQHGT4YTBTA57ZS3JU4YIGTLNYU7JQKUUZGPUQTM7I2JD4",
    "metadata-hash": "5zAUYEQ+kEKv8VCUNu17DMVNzpzRrF7X9JUr+OEfgp4=",
    "name": "Alice's Artwork 001",
    "name-b64": "QWxpY2UncyBBcnR3b3JrIDAwMQ==",
    "total": 1,
    "unit-name": "ALICE001",
    "unit-name-b64": "QUxJQ0UwMDE=",
    "url": "https://path/to/my/asset/details",
    "url-b64": "aHR0cHM6Ly9wYXRoL3RvL215L2Fzc2V0L2RldGFpbHM="
}
Asset ID: 1087
{
    "amount": 1,
    "asset-id": 1087,
    "is-frozen": false
}


## transferring asset without optin results in error

In [24]:
asset_id2="1112"
transfer_asset(m,receiver_pk,asset_id2)

AlgodHTTPError: TransactionPool.Remember: transaction DSOPJVUDJPFYIH7BYLZLKUI5SI23DTUMVZ3YGGELDDEVJNXXPZIA: asset 1112 missing from 4W45VZP6SMVYITTI7SJESMVSAJ6ATGGAQYZQCTRS22BSO4RWWPPEZMTIMQ

Get asset id

In [25]:
create_asset(m,receiver_pk)

--------------------------------------------
Creating Asset...
Asset Creation Transaction ID: XICD6G4MNXONEZGDZXUMOBBMDSLXGDNTD7KBH5C6Z4NV2NSZ2GQA
TXID:  XICD6G4MNXONEZGDZXUMOBBMDSLXGDNTD7KBH5C6Z4NV2NSZ2GQA
Result confirmed in round: 81
Asset ID: 1081
{
    "creator": "4W45VZP6SMVYITTI7SJESMVSAJ6ATGGAQYZQCTRS22BSO4RWWPPEZMTIMQ",
    "decimals": 0,
    "default-frozen": false,
    "manager": "4W45VZP6SMVYITTI7SJESMVSAJ6ATGGAQYZQCTRS22BSO4RWWPPEZMTIMQ",
    "metadata-hash": "5zAUYEQ+kEKv8VCUNu17DMVNzpzRrF7X9JUr+OEfgp4=",
    "name": "Alice's Artwork 001",
    "name-b64": "QWxpY2UncyBBcnR3b3JrIDAwMQ==",
    "reserve": "KOSJICYJYGMXQIQVVK55G2LQZMKX7QFPBC4VRMW2B4ROI4BKPW2POOFXTA",
    "total": 1,
    "unit-name": "ALICE001",
    "unit-name-b64": "QUxJQ0UwMDE=",
    "url": "https://path/to/my/asset/details",
    "url-b64": "aHR0cHM6Ly9wYXRoL3RvL215L2Fzc2V0L2RldGFpbHM="
}
Asset ID: 1081
{
    "amount": 1,
    "asset-id": 1081,
    "is-frozen": false
}


In [26]:
account_info = algod_client.account_info(receiver_pk)
# holding = None
# idx = 0
# for my_account_info in account_info['assets']:
#     scrutinized_asset = account_info['assets'][idx]
#     idx = idx + 1    
#     if (scrutinized_asset['asset-id'] == asset_id):
#         holding = True
#         break
account_info

{'address': 'KOSJICYJYGMXQIQVVK55G2LQZMKX7QFPBC4VRMW2B4ROI4BKPW2POOFXTA',
 'amount': 997000,
 'amount-without-pending-rewards': 997000,
 'apps-local-state': [],
 'apps-total-schema': {'num-byte-slice': 0, 'num-uint': 0},
 'assets': [{'amount': 0, 'asset-id': 1068, 'is-frozen': False},
  {'amount': 0, 'asset-id': 1074, 'is-frozen': False},
  {'amount': 1, 'asset-id': 1077, 'is-frozen': False}],
 'created-apps': [],
 'created-assets': [],
 'min-balance': 400000,
 'pending-rewards': 0,
 'reward-base': 0,
 'rewards': 0,
 'round': 81,
 'status': 'Offline',
 'total-apps-opted-in': 0,
 'total-assets-opted-in': 3,
 'total-created-apps': 0,
 'total-created-assets': 0}